<a href="https://colab.research.google.com/github/NVD1210/Recommender-System/blob/main/Question3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup a Spark environment**

In [ ]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# install spark (change the version number if needed)
!wget -q https://downloads.apache.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop3.2.tgz
# unzip the spark file to the current folder
!tar xf spark-3.1.3-bin-hadoop3.2.tgz

In [ ]:
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop3.2"
# install findspark using pip
!pip install -q findspark
!pip install pyspark==3.1.3



     |████████████████████████████████| 214.0 MB 8.6 kB/s 
     |████████████████████████████████| 198 kB 63.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.3-py2.py3-none-any.whl size=214463484 sha256=8c243438e6cd2ac5c8a7127ffd2feeb8f0a32f0663e53499f46e6026855db71a
  Stored in directory: /root/.cache/pip/wheels/ad/8e/49/44c110bb8e008d0778c6577d600d46047c6478ecca3f8f1517
Successfully built pyspark


In [ ]:
# !pip install -U --no-cache-dir gdown --pre
!pip install recommenders

     |████████████████████████████████| 335 kB 9.0 MB/s 
     |████████████████████████████████| 11.8 MB 32.3 MB/s 
     |████████████████████████████████| 310 kB 46.9 MB/s 
     |████████████████████████████████| 61 kB 6.7 MB/s 
     |████████████████████████████████| 4.0 MB 41.7 MB/s 
     |████████████████████████████████| 1.5 MB 37.3 MB/s 
     |████████████████████████████████| 12.4 MB 28.9 MB/s 
     |████████████████████████████████| 636 kB 65.6 MB/s 
     |████████████████████████████████| 197 kB 58.7 MB/s 
     |████████████████████████████████| 749 kB 49.7 MB/s 
     |████████████████████████████████| 10.9 MB 43.6 MB/s 
     |████████████████████████████████| 381 kB 63.8 MB/s 
     |████████████████████████████████| 895 kB 47.3 MB/s 
     |████████████████████████████████| 6.6 MB 33.7 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705357 sha256=a351548fe88699c91a0a86048365a6834c

In [ ]:
# set the environment path to find Recommenders
import sys
import warnings
import numpy as np
from numpy import longlong

import pyspark
from pyspark import SparkConf, SparkContext, sql

import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col
from pyspark.sql.functions import udf

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType
warnings.simplefilter(action='ignore', category=FutureWarning)

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))


System version: 3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]
Spark version: 3.1.3


# **MovieLens 10M Dataset**

**Mô tả Dataset**

*   MovieLens 10M movie ratings.
*   size: 63 MB.
*   Data: 10M và 100K tag applications.
*   Thu thập từ: 72,000 users cho 10,000 movies.
*   Released 1/2009.


## **Bước 1:** Load dữ liệu và chia mẫu train/test.

Dữ liệu sẽ được load dưới dạng rdd. Đây là định dạng dữ liệu chịu lỗi tốt và có khả năng phân tán linh hoạt trên nhiều cụm xử lý của spark. Vì những tiện ích này chúng thường được sử dụng trong các tính toán và biến đổi dữ liệu.

In [ ]:


# Creating a SparkSession in Python
# spark = SparkSession \
#     .builder \
#     .appName('ALS recommendation spark session') \
#     .getOrCreate()

spark = SparkSession.builder.master("local[4]")\
          .appName("StructureAPI")\
          .config('spark.driver.memory','5g')\
          .config('spark.submit.deployMode','client')\
          .config('spark.executor.memory','2g')\
          .config('spark.executor.cores',4)\
          .config('spark.yarn.queue','short')\
          .getOrCreate()

# lines = spark.read.text('data-ml-1m/ml-1m/ratings.dat').rdd
# parts = lines.map(lambda row: row.value.split('::'))

# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]), timestamp=int(p[3])))

# ratings = spark.createDataFrame(ratingsRDD)

# (training, testing) = ratings.randomSplit([0.8, 0.2])

In [ ]:
# top k items to recommend
TOP_K = 10


# Column names for the dataset
COL_USER = "userId"
COL_ITEM = "movieId"
COL_RATING = "rating"
COL_TIMESTAMP = "timestamp"

In [ ]:
# Note: The DataFrame-based API for ALS currently only supports integers for user and item ids.
schema = StructType(
    (
        StructField(COL_USER, IntegerType()),
        StructField(COL_ITEM, IntegerType()),
        StructField(COL_RATING, FloatType()),
        StructField(COL_TIMESTAMP, LongType()),
    )
)

# Select MovieLens data size: 100k, 1m, 10m, or 20m
data = movielens.load_spark_df(spark, size='10m', schema=schema)
# data.show()

100%|██████████| 64.0k/64.0k [00:02<00:00, 21.5kKB/s]


In [ ]:
def get_mat_sparsity(ratings):
    # Count the total number of ratings in the dataset
    count_nonzero = ratings.select("rating").count()

    # Count the number of distinct userIds and distinct movieIds
    total_elements = ratings.select("userId").distinct().count() * ratings.select("movieId").distinct().count()

    # Divide the numerator by the denominator
    sparsity = (1.0 - (count_nonzero *1.0)/total_elements)*100
    print("The ratings dataframe is ", "%.2f" % sparsity + "% sparse.")
    
get_mat_sparsity(data)

The ratings dataframe is  98.66% sparse.


In [ ]:
train, test = spark_random_split(data, ratio=0.8, seed=123)
print ("N train", train.cache().count())
print ("N test", test.cache().count())

N train 7997605
N test 2002449


## **Bước 2:** Xây dựng mô hình ALS trên pyspark.

### Xây dựng mô hình Recommendation

Sử dụng module *`pyspark.ml.recommendation`* ta có thể dễ dàng xây dựng một mô hình recommendation theo phương pháp ALS. 

Các tham số khi khởi tạo mô hình đó là:

*   maxIter: Số lượng vòng lặp tối đa
*   regParam: Hệ số kiểm soát regularization
*   userCol: tên cột chứa id của user
*   itemCol: tên cột chứa id của item
*   ratingCol: tên cột chứa giá trị của rating






In [ ]:
# # Xây dựng mô hình recommendation sử dụng thuật toán ALS trên tập dữ liệu huấn luyện
# als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", nonnegative=True)
# model = als.fit(training)

In [ ]:
header = {
    "userCol": COL_USER,
    "itemCol": COL_ITEM,
    "ratingCol": COL_RATING,
}

als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=False,
    seed=42,
    **header
)

## **Bước 3:** Evaluation Metrics

Took 75.85578419299998 seconds for training.


In [ ]:
with Timer() as train_time:
    model = als.fit(train)

print("Took {} seconds for training.".format(train_time.interval))

predictions = model.transform(test)

Took 83.46696679600001 seconds for training.


#### **RMSE**

Đánh giá mô hình thông qua chỉ số căn bậc 2 của trung bình sai số dự báo so với giá trị rating trên tập test, chỉ số RMSE.

In [ ]:
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)

In [ ]:
# print("\nRoot-mean-square error = " + str(rmse))

In [ ]:
rating_eval = SparkRatingEvaluation(test, predictions, col_user=COL_USER, col_item=COL_ITEM, 
                                    col_rating=COL_RATING, col_prediction="prediction")

print("Model:\tALS rating prediction",
      "RMSE:\t%f" % rating_eval.rmse(),
      "MAE:\t%f" % rating_eval.mae(),
      "Explained variance:\t%f" % rating_eval.exp_var(),
      "R squared:\t%f" % rating_eval.rsquared(), sep='\n')

Model:	ALS rating prediction
RMSE:	0.798771
MAE:	0.620187
Explained variance:	0.440936
R squared:	0.432845


#### **NDCG**

Took 4628.912535271 seconds for prediction.


In [ ]:
with Timer() as test_time:

    # Get the cross join of all user-item pairs and score them.
    users = train.select(COL_USER).distinct()
    items = train.select(COL_ITEM).distinct()
    user_item = users.crossJoin(items)
    dfs_pred = model.transform(user_item)

    # Remove seen items.
    dfs_pred_exclude_train = dfs_pred.alias("pred").join(
        train.alias("train"),
        (dfs_pred[COL_USER] == train[COL_USER]) & (dfs_pred[COL_ITEM] == train[COL_ITEM]),
        how='outer'
    )

    top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train[f"train.{COL_RATING}"].isNull()) \
        .select('pred.' + COL_USER, 'pred.' + COL_ITEM, 'pred.' + "prediction")

    # In Spark, transformations are lazy evaluation
    # Use an action to force execute and measure the test time 
    # top_all.cache().count()

print("Took {} seconds for prediction.".format(test_time.interval))

Took 0.20226422999996885 seconds for prediction.


In [ ]:
rank_eval = SparkRankingEvaluation(test, top_all, k = TOP_K, col_user=COL_USER, col_item=COL_ITEM, 
                                    col_rating=COL_RATING, col_prediction="prediction", 
                                    relevancy_method="top_k")

In [ ]:
print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "NDCG:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@K:\t%f" % rank_eval.precision_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	ALS
Top K:	10
MAP:	0.000635
NDCG:	0.005446
Precision@K:	0.006864
Recall@K:	0.003564


#### **HIT RATE**

In [ ]:
# Tạo ra top 10 bộ phim khuyến nghị cho mỗi một user từ tập test
TopPredict = model.recommendForUserSubset(test, 10)
top_predict_dict = {}
for row in TopPredict.collect():
  top_predict_dict[row['userId']] = [ movie_['movieId']  for movie_ in row['recommendations']]

In [ ]:
# # See how often we recommended a movie the user actually rated
def HitRate(topPredicted, Predictions):
    hits = 0
    total = 0
 # For each left-out rating
    for leftOut in Predictions.collect():
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        # Is it in the predicted top 10 for this user?
        hit = False
        for movieID in topPredicted[int(userID)]:
            # print(leftOutMovieID, "    ", movieID)
            if (int(leftOutMovieID) == int(movieID)):
                hit = True
                break
        if (hit):
            hits += 1

        total += 1

    # # Compute overall precision
    return hits/total

In [ ]:
print("\nHit Rate: ", HitRate(top_predict_dict, predictions))


Hit Rate:  0.001974624526694387


## **Bước 4:** Khuyến nghị phim cho người dùng và khuyến nghị người dùng cho phim.

In [ ]:
# Tạo ra top 10 bộ phim khuyến nghị cho mỗi một user
userRecs = model.recommendForAllUsers(10)

In [ ]:
userRecs.show(truncate=False)

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                                   |
+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|148   |[{25975, 5.9770083}, {33264, 5.6193523}, {64408, 5.5783195}, {57499, 5.4064713}, {32657, 5.25846}, {7144, 5.2430873}, {53883, 5.2336335}, {4454, 5.152204}, {720, 5.1415186}, {8120, 5.110942}]   |
|463   |[{32657, 5.0905037}, {25975, 4.9146233}, {1198, 4.693926}, {260, 4.6364336}, {1196, 4.558309}, {64408, 4.4495726}, {33264, 4.4361687}, {318, 4.4339046}, {1036, 4.423878}, {7153

In [ ]:
# Tạo ra top 10 user khuyến nghị cho mỗi một bộ phim
movieRecs = model.recommendForAllItems(10)

In [ ]:
movieRecs.show(truncate=False)

+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|movieId|recommendations                                                                                                                                                                                      |
+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1580   |[{62350, 5.663908}, {9269, 5.6448183}, {31669, 5.3394833}, {45919, 5.331946}, {54749, 5.2863727}, {55934, 5.2701187}, {28132, 5.2676635}, {65873, 5.261842}, {37013, 5.225602}, {60907, 5.2165236}]  |
|4900   |[{45771, 6.199051}, {48469, 5.901399}, {61093, 5.7784047}, {14858, 5.715238}, {25958, 5.550952}, {2117, 5.464593}, {12485, 5.4526854}, {1716, 5.447732}, {10307

# Stop Spark

In [ ]:
spark.stop()